<a href="https://colab.research.google.com/github/MusabUmama/Movie_Recommendation_system/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collaborative Movie Filtering

**The Dataset**

There are 4 datasets including movies, ratings, tags and links.


* Movies Dataset: This dataset contains information about movies, including movie
IDs, titles, and genres.

* Ratings Dataset: This dataset contains user ratings for movies, including user IDs, movie IDs, ratings, and timestamps.

* Tags Dataset: This dataset contains user-generated tags for movies, including user IDs, movie IDs, tags, and timestamps.

* Links Dataset: This dataset contains links between movie IDs in the dataset and external databases (IMDb and TMDB).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-d5df0069828e>", line 2, in <cell line: 2>
    drive.mount('/content/drive')
  File "/usr/local/lib/python3.10/dist-packages/google/colab/drive.py", line 103, in mount
    return _mount(
  File "/usr/local/lib/python3.10/dist-packages/google/colab/drive.py", line 132, in _mount
    _message.blocking_request(
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_message.py", line 176, in blocking_request
    return read_reply_from_input(request_id, timeout_sec)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_message.py", line 103, in read_reply_from_input
    raise MessageError(reply['error'])
google.colab._message.MessageError: Error: credential propagation was unsuccessful

During handling of the above exception, another exception occurred:

Traceb

TypeError: ignored

# **User-based Movie Filtering**

* The system makes recommendations based on similar users



In [ ]:
import pandas as pd

In [ ]:
# Importing the datasets
movies_data = pd.read_csv("/content/movies.csv")
ratings_data = pd.read_csv("/content/ratings.csv")
tags_data = pd.read_csv("/content/tags.csv")

In [ ]:
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
# Deleting the null rows
movies_data.dropna(inplace=True)
ratings_data.dropna(inplace=True)
tags_data.dropna(inplace=True)

# Deleting duplicate rows
movies_data.drop_duplicates(subset='movieId', keep='first')

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
movies_data.dtypes

movieId     int64
title      object
genres     object
dtype: object

In [ ]:
ratings_data.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [ ]:
tags_data.dtypes

userId        int64
movieId       int64
tag          object
timestamp     int64
dtype: object

In [ ]:
# Merging the Ratings and Tags datasets based on 'movieId' and 'userId'
user_interactions_data = pd.merge(ratings_data, tags_data, on=['userId', 'movieId'], how='outer')

# Merging the user_interactions dataframe with the Movies dataset based on 'movieId' to create merged data frame
merged_data = pd.merge(user_interactions_data, movies_data, on='movieId', how='left')

In [ ]:
merged_data.head()

,userId,movieId,rating,timestamp_x,tag,timestamp_y,title,genres
0,1,1,4.0,964982703.0,NaN,NaN,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247.0,NaN,NaN,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224.0,NaN,NaN,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815.0,NaN,NaN,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931.0,NaN,NaN,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


The merged data frame has been created using relevant features from all the datasets.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Creating a user-item interaction matrix
interaction_matrix = pd.pivot_table(merged_data, values='rating', index='userId', columns='movieId', fill_value=0)

In [ ]:
# Calculating the user similarity scores using cosine similarity
user_similarity_scores = cosine_similarity(interaction_matrix)

In [ ]:
# Creating a data frame to store user similarity scores
user_similarity_data = pd.DataFrame(user_similarity_scores, index=interaction_matrix.index, columns=interaction_matrix.index)

# Replacing the diagonal values with zeros (self-similarity scores)
user_similarity_data.values[[range(user_similarity_data.shape[0])]*2] = 0

In [ ]:
interaction_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0


In [ ]:
user_similarity_data.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import numpy as np

In [ ]:
# Choosing a target user
target_user_id = 300

The user with ID no. 300 has been choosen as the target user

In [ ]:
# Finding the most similar users to the target user in descending order
similar_users = user_similarity_data[target_user_id].sort_values(ascending=False)

In [ ]:
similar_users

userId
1      0.0
410    0.0
403    0.0
404    0.0
405    0.0
      ... 
205    0.0
206    0.0
207    0.0
208    0.0
610    0.0
Name: 300, Length: 610, dtype: float64

In [ ]:
# Geting the movies that the user has not rated
rated_movies = interaction_matrix.loc[target_user_id]
unrated_movies = rated_movies[rated_movies == 0].index

In [ ]:
# Calculating the weighted sum of ratings by similar users for unrated movies
unrated_movie_scores = interaction_matrix.loc[similar_users.index, unrated_movies].T.dot(similar_users)

In [ ]:
# Sorting the recommended movies by their scores in descending order
recommended_movies_id = unrated_movie_scores.sort_values(ascending=False)

In [ ]:
recommended_movies_id

movieId
1         0.0
53161     0.0
53318     0.0
53322     0.0
53326     0.0
         ... 
4390      0.0
4392      0.0
4393      0.0
4394      0.0
193609    0.0
Length: 9692, dtype: float64

In [ ]:
recommended_movie_titles = merged_data[merged_data['movieId'].isin(recommended_movies_id.index)]['title']

In [ ]:
recommended_movie_titles

0                                          Toy Story (1995)
1                                   Grumpier Old Men (1995)
2                                               Heat (1995)
3                               Seven (a.k.a. Se7en) (1995)
4                                Usual Suspects, The (1995)
                                ...                        
102878                                    Score, The (2001)
102880                                     Daredevil (2003)
102881                                     Daredevil (2003)
102882    Mary Shelley's Frankenstein (Frankenstein) (1994)
102883                               Shame (Skammen) (1968)
Name: title, Length: 98835, dtype: object

Some movies are recommended multiple times in the list due to multiple similar users rating those movies highly.

In [ ]:
# List to store unique movies
unique_movies = []

In [ ]:
# Adding the movies to unique list
for movie_id in recommended_movie_titles:
    if movie_id not in unique_movies:
        unique_movies.append(movie_id)

In [ ]:
# Top 10 movies recommended
print("Top 10 recommendations:\n")
for movie in range(10):
  print(unique_movies[movie])

Top 10 recommendations:

Toy Story (1995)
Grumpier Old Men (1995)
Heat (1995)
Seven (a.k.a. Se7en) (1995)
Usual Suspects, The (1995)
From Dusk Till Dawn (1996)
Bottle Rocket (1996)
Braveheart (1995)
Rob Roy (1995)
Canadian Bacon (1995)


# **Item-based movie filtering**

* Item-Based Collaborative Filtering recommends items to users based on the similarity between items rather than users.


All the datasets and libraries imported in user based filtering is being used here.

In [ ]:
# Merging the Movies and Ratings datasets based on movieId
movie_ratings_data = pd.merge(ratings_data, movies_data, on='movieId')